# Watershed Distance Transform for 2D Data
---
Implementation of papers:

[Deep Watershed Transform for Instance Segmentation](http://openaccess.thecvf.com/content_cvpr_2017/papers/Bai_Deep_Watershed_Transform_CVPR_2017_paper.pdf)

[Learn to segment single cells with deep distance estimator and deep cell detector](https://arxiv.org/abs/1803.10829)

In [2]:
import os
import errno

import numpy as np

import deepcell
from tensorflow.python import keras


In [3]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

n_epoch = 60  # Number of training epochs
test_size = .10  # % of data saved as test
norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Sample mode settings
batch_size = 64  # number of images per batch (should be 2 ^ n)
win = (receptive_field - 1) // 2  # sample window size
balance_classes = True  # sample each class equally
max_class_samples = 1e6  # max number of samples per class

# Transformation settings
transform = 'watershed'
distance_bins = 4  # number of distance "classes"
erosion_width = 1  # erode edges

In [3]:
base_name = 'Point1_12_18_3X_interior_border_border'
npz_name = '/data/npz_data/' + base_name + ".npz"
MODEL_DIR = '/data/models/' + '20190606_params'

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

LOG_DIR = '/data/logs'


fgbg_model_name = base_name + "_fgbg"
sample_model_name = base_name + "_watershed_64_filters_400_densefilters_balanced"

### Load the Training Data

In [4]:
CHANNEL_AXIS = 3
training_data = np.load(npz_name)

X, y = training_data["X"], training_data["y"]
print("X.shape: {} & y.shape: {}".format(X.shape, y.shape))
#X = X[:, :, :, 0:1]

X.shape: (3, 1024, 1024, 3) & y.shape: (3, 1024, 1024, 1)


### First, create a foreground/background separation model

#### Instantiate the fgbg model

In [23]:
from deepcell import model_zoo

fgbg_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=2)

#### Train the fgbg model

In [25]:
from deepcell.training import train_model_sample

fgbg_model = train_model_sample(
    model=fgbg_model,
    dataset=npz_name,
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    window_size=(win, win),
    batch_size=128,
    transform='fgbg',
    n_epoch=5,
    balance_classes=balance_classes,
    max_class_samples=1000000,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=1)

using all data as training data
Using class weights of {0: 1.0, 1: 1.0}
X_train shape: (3, 1024, 1024, 1)
y_train shape: (3, 1024, 1024, 1)
Output Shape: (None, 2)
Number of Classes: 2
Training on 1 GPUs
the max_class_samples per image is 333333
analyzing image 0
the least represented class has 331425 examples
analyzing class 0
downsampling from 595944 examples per class
analyzing class 1
downsampling from 331425 examples per class
analyzing image 1
the least represented class has 421168 examples
max_class_samples is less than the smalleset class, downsampling all classes
analyzing class 0
downsampling from 506201 examples per class
analyzing class 1
downsampling from 421168 examples per class
analyzing image 2
the least represented class has 434941 examples
max_class_samples is less than the smalleset class, downsampling all classes
analyzing class 0
downsampling from 434941 examples per class
analyzing class 1
downsampling from 492428 examples per class
running model without validati

### Next, Create a model for the watershed energy transform

#### Instantiate the distance transform model

In [5]:
from deepcell import model_zoo
from deepcell.training import train_model_sample

watershed_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=distance_bins,
    n_conv_filters=64,
    n_dense_filters=400)

In [6]:
sample_model_name
#watershed_model.load_weights('/data/models/20190606_params/Point1_12_18_3X_interior_border_border_watershed_128_filters_balanced_epoch_40.h5')

'Point1_12_18_3X_interior_border_border_watershed_64_filters_400_densefilters_balanced'

#### Train the model

In [ ]:
watershed_model = train_model_sample(
    model=watershed_model,
    dataset=npz_name, 
    model_name=sample_model_name,
    test_size=test_size,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=60,
    window_size=(win, win),
    transform=transform,
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    balance_classes=True,
    max_class_samples=180000,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=10)

using all data as training data
Using class weights of {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0}
X_train shape: (3, 1024, 1024, 3)
y_train shape: (3, 1024, 1024, 1)
Output Shape: (None, 4)
Number of Classes: 4
Training on 1 GPUs
the max_class_samples per image is 60000
analyzing image 0
the least represented class has 30524 examples
analyzing class 0
downsampling from 739622 examples per class
analyzing class 1
downsampling from 95868 examples per class
analyzing class 2
downsampling from 61355 examples per class
analyzing class 3
downsampling from 30524 examples per class
analyzing image 1
the least represented class has 37791 examples
analyzing class 0
downsampling from 687593 examples per class
analyzing class 1
downsampling from 121406 examples per class
analyzing class 2
downsampling from 80579 examples per class
analyzing class 3
downsampling from 37791 examples per class
analyzing image 2
the least represented class has 44239 examples
analyzing class 0
downsampling from 646714 examples p

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7034/7034 [==============================] - 540s 77ms/step - loss: 0.5484 - acc: 0.7481
Epoch 26/60
7034/7034 [==============================] - 537s 76ms/step - loss: 0.5449 - acc: 0.7502
Epoch 27/60
7034/7034 [==============================] - 543s 77ms/step - loss: 0.5414 - acc: 0.7514
Epoch 28/60
7034/7034 [==============================] - 541s 77ms/step - loss: 0.5385 - acc: 0.7529
Epoch 29/60
7034/7034 [==============================] - 540s 77ms/step - loss: 0.5364 - acc: 0.7546
Epoch 30/60
7033/7034 [============================>.] - ETA: 0s - loss: 0.5331 - acc: 0.7560
Epoch 00030: saving model to /data/models/20190606_params/Point1_12_18_3X_interior_border_border_watershed_64_filters_400_densefilters_balanced_epoch_30.h5
7034/7034 [==============================] - 539s 77ms/step - loss: 0.5330 - acc: 0.7560
Epoch 31/60
7034/7034 [==============================] - 533s 76ms/step - loss: 0.5305 - acc: 0.7572
Epoch 32/60
7034/7034 [==============================] - 534s 76ms/